In [ ]:
import pandas as pd
import pyodbc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
# nltk.download('stopwords')

server = 'bg-aws-dw1p'
database = 'ADS_SF'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                        SERVER=' + server + '; \
                        DATABASE=' + database +';\
                        Trusted_Connection=yes;')

query = '''with 
Recommended as (
    select distinct 
        IfYouWereToRecommendUs__c as Text,
        1 as IsRecommendedFlag
from ODSSF.dbo.NPSSurvey__c
    where IfYouWereToRecommendUs__c is not null),
NotRecommended as (
    select distinct
        WhyWouldntYouRecommendUs__c as Text,
        0 as IsRecommendedFlag
from ODSSF.dbo.NPSSurvey__c
    where WhyWouldntYouRecommendUs__c is not null)
    
select * from Recommended
UNION ALL
select * from NotRecommended'''

df = pd.read_sql_query(query,cnxn)

cnxn.close()

pd.set_option('max_colwidth',600)

In [ ]:
identifier1 = 1
identifier0 = 0

a = df[df['IsRecommendedFlag']==1]['Text']
a = a.str.replace(r'[^\w\s]+', ' ')
a2 = a.apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stopwords.words('english')]))
a3 = a2.tolist()
a3 = a2.str.split()

aa = pd.DataFrame(a3)
aa['IsRecommendedFlag'] = identifier1
aa = aa.explode('Text')

b = df[df['IsRecommendedFlag']==0]['Text']
b = b.str.replace(r'[^\w\s]+', ' ')
b2 = b.apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stopwords.words('english')]))
b3 = b2.tolist()
b3 = b2.str.split()

bb = pd.DataFrame(b3)
bb['IsRecommendedFlag'] = identifier0
bb = bb.explode('Text')

frames = [aa, bb]
final = pd.concat(frames)

final.columns = ['Text_Cleaned', 'IsRecommendedFlag']
final = final[~final['Text_Cleaned'].isnull()]
final = final[~final['Text_Cleaned'].str.isdigit()]
final = final[~final['Text_Cleaned'].str.isalpha() == False]

In [ ]:
print(final)

In [ ]:
# DJ edits:

def clean_text(text):
    # Replace non-alphanumeric characters with space
    text = text.str.replace(r'[^\w\s]+', ' ')
    # Remove stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.lower().split() if word not in stopwords.words('english')]))
    # Remove null values, digits, and strings that are not alphabetic characters
    text = text[~text.isnull() & ~text.str.isdigit() & (text.str.isalpha() == False)]
    
    return text.str.split()

def process_dataframe(df, flag):
    # Clean and process text
    processed = clean_text(df[df['IsRecommendedFlag'] == flag]['Text'])
        
    # Convert to DataFrame and add flag
    result = pd.DataFrame({'Text_Cleaned': processed, 'IsRecommendedFlag': flag})
        
    return result.explode('Text_Cleaned')

recommended = process_dataframe(df, 1)
notrecommended = process_dataframe(df, 0)

# Concatenate the results
final = pd.concat([recommended, notrecommended], ignore_index=True)